# Causas externas de morbidade e de mortalidade: Acidentes de Transporte


A CID-10 é a Classificação Internacional de Doenças [(International Statistical Classification of Diseases and Related Health Problems – ICD](https://icd.who.int/browse10/2010/en), está em sua décima revisão, feita em 2010. 
Essa classificação também abrange as causas externas de mortalidade, como acidentes de trânsito. E visto que o CID-10 também é usado nos dados do SIM, precisamos saber a descrição de cada código a fim de termos maior entendimento sobre a mortalidade no trânsito.

A partir da página [CID-10 Capítulo XX: Causas externas de morbidade e de mortalidade](https://pt.wikipedia.org/wiki/CID-10_Cap%C3%ADtulo_XX:_Causas_externas_de_morbidade_e_de_mortalidade) disponível no Wikipédia, fizemos a extração dos códigos e descrições.

In [1]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
import lxml
pd.set_option('display.max_colwidth', None)

Os dados disponíveis em [A Classificação Internacional de Doenças CID-10 do Mobilidade Ativa]("http://mobilidadeativa.org.br/cid-10/") fornece a descrição das faixas de código. O ideal seria ter a informação no formato mais granular possível.

In [2]:
causes = pd.read_html("http://mobilidadeativa.org.br/cid-10/")
causes[0]

,0,1
0,Categoria CID-10,Descrição
1,V01-V09,Pedestre traumatizado em acidente de transporte
2,V10-V19,Ciclista traumatizado em acidente de transporte
3,V20-V29,Motociclista traumatizado em acidente de transporte
4,V30-V39,Ocupante de triciclo motorizado traumatizado em acidente de transporte
5,V40-V49,Ocupante de automóvel traumatizado em acidente de transporte
6,V50-V59,Ocupante de caminhonete traumatizado em acidente de transporte
7,V60-V69,Ocupante de veículo de transporte pesado traumatizado em acidente
8,V70-V79,Ocupante de ônibus traumatizado em acidente de transporte
9,V80-V89,Outros acidentes de transporte terrestre


Vamos tentar a partir da página [CID-10 Capítulo XX: Causas externas de morbidade e de mortalidade do Wikipédia](https://pt.wikipedia.org/wiki/CID-10_Cap%C3%ADtulo_XX:_Causas_externas_de_morbidade_e_de_mortalidade)

In [3]:
page = requests.get("https://pt.wikipedia.org/wiki/CID-10_Cap%C3%ADtulo_XX:_Causas_externas_de_morbidade_e_de_mortalidade")
page = bs(page.text, 'html.parser')

In [4]:
causes = []

for ul in page.findAll('ul'):
    for li in ul.findAll('li'):
        if li.text[0:3] in ['(V0','(V1', '(V2', '(V3', '(V4', '(V5', '(V6', '(V7', '(V8', '(V9']:
            causes.append(li.text)
            if len(li.findAll('ul')) != 0:
                for ul in li.findAll('ul'):
                    for li in ul.findAll('li'):
                        causes.append(li.text)

len(causes)

625

In [61]:
causes_df = pd.DataFrame({'causes': causes})
causes_df.head()

,causes
0,(V01) Pedestre traumatizado em colisão com um veículo a pedal
1,(V02) Pedestre traumatizado em colisão com um veículo a motor de duas ou três rodas
2,"(V03) Pedestre traumatizado em colisão com um automóvel (carro), “pick up” ou caminhonete"
3,(V04) Pedestre traumatizado em colisão com um veículo de transporte pesado ou com um ônibus
4,(V05) Pedestre traumatizado em colisão com trem (comboio) ou um veículo ferroviário


In [62]:
causes_df['causes'] = causes_df['causes'].str.rstrip().str.lstrip().str.lstrip("(")
causes_df.head()

,causes
0,V01) Pedestre traumatizado em colisão com um veículo a pedal
1,V02) Pedestre traumatizado em colisão com um veículo a motor de duas ou três rodas
2,"V03) Pedestre traumatizado em colisão com um automóvel (carro), “pick up” ou caminhonete"
3,V04) Pedestre traumatizado em colisão com um veículo de transporte pesado ou com um ônibus
4,V05) Pedestre traumatizado em colisão com trem (comboio) ou um veículo ferroviário


In [63]:
causes_df['code'] = causes_df['causes'].str.split(")").str[0]
causes_df.head()

,causes,code
0,V01) Pedestre traumatizado em colisão com um veículo a pedal,V01
1,V02) Pedestre traumatizado em colisão com um veículo a motor de duas ou três rodas,V02
2,"V03) Pedestre traumatizado em colisão com um automóvel (carro), “pick up” ou caminhonete",V03
3,V04) Pedestre traumatizado em colisão com um veículo de transporte pesado ou com um ônibus,V04
4,V05) Pedestre traumatizado em colisão com trem (comboio) ou um veículo ferroviário,V05


In [66]:
causes_df['causes'] = causes_df['causes'].str.split(")").str[1] + \
                        causes_df['causes'].str.split(")").str[2].apply(lambda x: ")" + x if pd.notna(x) else "")

causes_df.head()

,causes,code
0,Pedestre traumatizado em colisão com um veículo a pedal,V01
1,Pedestre traumatizado em colisão com um veículo a motor de duas ou três rodas,V02
2,"Pedestre traumatizado em colisão com um automóvel (carro), “pick up” ou caminhonete",V03
3,Pedestre traumatizado em colisão com um veículo de transporte pesado ou com um ônibus,V04
4,Pedestre traumatizado em colisão com trem (comboio) ou um veículo ferroviário,V05


In [67]:
causes_df['causes'] = causes_df['causes'].str.rstrip().str.lstrip()
causes_df.head()

,causes,code
0,Pedestre traumatizado em colisão com um veículo a pedal,V01
1,Pedestre traumatizado em colisão com um veículo a motor de duas ou três rodas,V02
2,"Pedestre traumatizado em colisão com um automóvel (carro), “pick up” ou caminhonete",V03
3,Pedestre traumatizado em colisão com um veículo de transporte pesado ou com um ônibus,V04
4,Pedestre traumatizado em colisão com trem (comboio) ou um veículo ferroviário,V05
